In [1]:
import sys

sys.path.append('..')

from LoadData2 import loadNbackEmoidAgesScansAndGenders, loadMeta

pncDir = '../../PNC_Good'

keys, nbackTs, emoidTs, ages = loadNbackEmoidAgesScansAndGenders(loadMeta(f'{pncDir}/MegaMeta3.pkl'))

print(nbackTs.shape)
print(emoidTs.shape)
print(ages.shape)
print(ages[0:10])

(650, 264, 231)
(650, 264, 210)
(650,)
[ 9.66666667 15.58333333 15.83333333 20.16666667 11.41666667 11.58333333
  9.41666667 17.5        15.25        9.58333333]


In [2]:
# Get FC and convert to torch

from LoadData2 import getFC
import torch

nbackP = getFC(nbackTs)
emoidP = getFC(emoidTs)

nbackP_t = torch.from_numpy(nbackP).reshape(650,264*264).float().cuda()
emoidP_t = torch.from_numpy(emoidP).reshape(650,264*264).float().cuda()
feat_t = torch.cat([nbackP_t, emoidP_t], dim=1)
ages_t = torch.from_numpy(ages).float().cuda()

print(nbackP_t.shape)
print(emoidP_t.shape)
print(ages_t.shape)

torch.Size([650, 69696])
torch.Size([650, 69696])
torch.Size([650])


In [4]:
import CG
import numpy as np

for i in range(4):
    si = i*100
    ti = (i+1)*100
    ei = (i+2)*100
    print(f'{si} {ti} {ei}')
    trainFeat = feat_t[si:ti]
    testFeat = feat_t[ti:ei]
    feat = feat_t[si:ei]
    trainA = CG.cosineSimilarityAdjacency(trainFeat, trainFeat)
    A = CG.cosineSimilarityAdjacency(feat, feat)
    trainLabels = ages_t[si:ti]
    testLabels = ages_t[ti:ei]
    testLabels_np = testLabels.detach().cpu().numpy()
    print('Least squares')
    w, _, _, _ = torch.linalg.lstsq(trainFeat, trainLabels)
    pred = testFeat@w
    pred = pred.detach().cpu().numpy()
    print(np.mean((pred-testLabels_np)**2)**0.5)
    print('GCN')
    gcn = CG.createGCN([2*264*264,100,1], torch.nn.MSELoss())
    CG.trainGCN(gcn, trainA, trainFeat, trainLabels, nEpochs=2000, verbose=False)
    pred = gcn([A, feat]).flatten()[(ti-si):].detach().cpu().numpy()
    print(np.mean((pred-testLabels_np)**2)**0.5)
    print('CG')
    cg = CG.createCG([4*264*264,100,1], torch.nn.MSELoss())
    CG.trainCG(cg, trainFeat, trainLabels, verbose=False)
    wp,wn = CG.evalCG(cg, trainFeat, trainLabels, testFeat, verbose=False)
    print(np.mean((wp-testLabels_np)**2)**0.5)
    print(np.mean((wn-testLabels_np)**2)**0.5)


0 100 200
Least squares
2.874002698330286
GCN
2.6527987330098917
CG
2.5719188081101687
2.57463502058566
100 200 300
Least squares
2.6585843767913944
GCN
2.593864162644314
CG
2.5260607560592976
2.526723248333008
200 300 400
Least squares
2.771158576940523
GCN
2.5990703204432792
CG
2.5293105577922925
2.526166410079507
300 400 500
Least squares
2.7200521222898715
GCN
2.609467830263047
CG
2.5216263790386306
2.522306430264847
